# An Optimal Selection of Features for Gait Recognition

### Library Imports

In [1]:
import numpy as np
import cv2
import os
import pickle
from scipy.io import mmread
import scipy.io
# import fishervectors

### Main Function

In [47]:
runCode()


Subject Folder: fyc

Sequence Folder: fyc/00_1

Video Path: fyc00_1.avi

Videos\fyc\00_1\fyc00_1.avi
Video Path: C:\Users\LENOVO\Desktop\AIM\Videos\fyc\00_1\fyc00_1.avi

Using cv2 to load video
Loaded video in 0.12 seconds

Now extracting HOG features. Timings below include loading the video (as in our paper):


error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'compute'
> Overload resolution failed:
>  - HOGDescriptor.compute() takes at most 4 arguments (5 given)
>  - HOGDescriptor.compute() takes at most 4 arguments (5 given)


### Load Video

In [38]:
import cv2

def loadVideo(videoPath):
    import os
    import time

    if not os.path.exists(videoPath):
        raise ValueError(f'Video file "{videoPath}" does not exist')

    print('Using cv2 to load video')
    start_time = time.time()
    cap = cv2.VideoCapture(videoPath)
    
    # Read the frames of the video
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    
    vid = frames
    videoReadTime = time.time() - start_time
    print(f'Loaded video in {videoReadTime:.2f} seconds')

    return vid, videoReadTime


In [46]:
import cv2
import numpy as np

def Video2DenseHOFVolumes(video, blockSize, numBlocks, numOr, flowMethod):
    # Initialize lists to store HOF descriptors and their corresponding information
    hofDesc = []
    hofInfo = []

    # Iterate over the video frames
    for frame_idx in range(1, len(video)):
        # Convert frames to grayscale
        prev_gray = cv2.cvtColor(video[frame_idx - 1], cv2.COLOR_BGR2GRAY)
        curr_gray = cv2.cvtColor(video[frame_idx], cv2.COLOR_BGR2GRAY)

        # Compute optical flow
        flow = computeOpticalFlow(prev_gray, curr_gray, flowMethod)

        # Compute HOF descriptors
        hof = cv2.HOFDescriptor()
        hofDesc_, hofInfo_ = hof.compute(flow, orientations=numOr, pixels_per_cell=(blockSize, blockSize), cells_per_block=(blockSize, blockSize))

        # Reshape the HOF descriptors into a volume
        hofDesc_ = hofDesc_.reshape(numBlocks, numBlocks, numOr)

        # Store the HOF descriptors and their information
        hofDesc.append(hofDesc_)
        hofInfo.append(hofInfo_)

    return hofDesc, hofInfo

def computeOpticalFlow(prev_gray, curr_gray, flowMethod):
    if flowMethod == 'dense':
        # Compute dense optical flow using Farneback method
        flow = cv2.calcOpticalFlowFarneback(prev_gray, curr_gray, None, pyr_scale=0.5, levels=3, winsize=15, iterations=3, poly_n=5, poly_sigma=1.1, flags=0)
    elif flowMethod == 'sparse':
        # Create sparse optical flow object using Lucas-Kanade method
        lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
        feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
        p0 = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)

        # Compute sparse optical flow using Lucas-Kanade method
        p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, p0, None, **lk_params)

        # Select good points
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # Create empty flow array
        flow = np.zeros_like(prev_gray)

        # Compute optical flow vectors
        flow_vectors = good_new - good_old

        # Assign optical flow vectors to flow array
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            flow = cv2.line(flow, (a, b), (c, d), (0, 255, 0), 1)
            flow = cv2.circle(flow, (a, b), 2, (0, 0, 255), -1)
    else:
        raise ValueError(f"Invalid flowMethod '{flowMethod}'. Valid options are 'dense' and 'sparse'.")

    return flow


### Extract Features

In [40]:
def extractFeatures(vid, videoReadTime):
    import time

    # Parameters
    blockSize = [16, 16, 6]
    numBlocks = [2, 2, 1]
    numOr = 9
    flowMethod = 'Horn-Schunck'

    # For-loop over the sampling rate for HOG
    print('\nNow extracting HOG features. Timings below include loading the video (as in our paper):')
    hogDesc = []
    hogInfo = []
    extractionTimeHOG = []
    
    vid = np.array(vid)
    for frameSampleRate in [1, 2, 3, 6]:
        start_time = time.time()
        # Subsample framerate of video
        sampledVid = vid[:, :, 0::frameSampleRate]

        # Get correct number of frames per block
        blockSize[2] = 6 / frameSampleRate

        # Get HOG descriptors
        hogDesc_, hogInfo_ = Video2DenseHOGVolumes(sampledVid, blockSize, numBlocks, numOr)
        hogDesc.append(hogDesc_)
        hogInfo.append(hogInfo_)

        # Print statistics
        extractionTimeHOG.append(time.time() - start_time)
        totalDescriptorTime = extractionTimeHOG[-1] + videoReadTime
        print('HOG: frames/block: {} sample rate: {} sec/vid: {:.2f} frame/sec: {:.2f}'.format(
            blockSize[2], frameSampleRate, totalDescriptorTime, vid.shape[2] / totalDescriptorTime))

    # For-loop over the sampling rate for HOF
    print('\nNow extracting HOF features. Timings below include loading the video (as in our paper):')
    hofDesc = []
    hofInfo = []
    extractionTimeHOF = []
    for frameSampleRate in [1, 2, 3, 6]:
        start_time = time.time()
        # Subsample framerate of video
        sampledVid = vid[:, :, 0::frameSampleRate]

        # Get correct number of frames per block
        blockSize[2] = 6 / frameSampleRate

        # Get HOF descriptors
        hofDesc_, hofInfo_ = Video2DenseHOFVolumes(sampledVid, blockSize, numBlocks, numOr, flowMethod)
        hofDesc.append(hofDesc_)
        hofInfo.append(hofInfo_)

        # Print statistics
        extractionTimeHOF.append(time.time() - start_time)
        totalDescriptorTime = extractionTimeHOF[-1] + videoReadTime
        print('HOF: frames/block: {} sample rate: {} sec/vid: {:.2f} frame/sec: {:.2f}'.format(
            blockSize[2], frameSampleRate, totalDescriptorTime, vid.shape[2] / totalDescriptorTime))

    # For-loop over the sampling rate for MBH
    print('\nNow extracting MBH features. Timings below include loading the video (as in our paper):')
    MBHRowDesc = []
    MBHColDesc = []
    mbhInfo = []
    extractionTimeMBH = []
    for frameSampleRate in [1, 2, 3, 6]:
        start_time = time.time()
        # Subsample framerate of video
        sampledVid = vid[:, :, 0::frameSampleRate]

        # Get correct number of frames per block
        blockSize[2] = 6 / frameSampleRate

        # Get MBH descriptors
        MBHRowDesc_, MBHColDesc_, mbhInfo_ = Video2DenseMBHVolumes(sampledVid, blockSize, numBlocks, numOr, flowMethod)
        MBHRowDesc.append(MBHRowDesc_)
        MBHColDesc.append(MBHColDesc_)
        mbhInfo.append(mbhInfo_)

        # Print statistics
        extractionTimeMBH.append(time.time() - start_time)
        totalDescriptorTime = extractionTimeMBH[-1] + videoReadTime
        print('MBH: frames/block: {} sample rate: {} sec/vid: {:.2f} frame/sec: {:.2f}'.format(
            blockSize[2], frameSampleRate, totalDescriptorTime, vid.shape[2] / totalDescriptorTime))

    # For-loop over the sampling rate for HMG
    print('\nNow extracting HMG features. Timings below include loading the video (as in our paper):')
    hmgDesc = []
    hmgInfo = []
    extractionTimeHMG = []
    for frameSampleRate in [1, 2, 3, 6]:
        start_time = time.time()
        # Subsample framerate of video
        sampledVid = vid[:, :, 0::frameSampleRate]

        # Get correct number of frames per block
        blockSize[2] = 6 / frameSampleRate

        # Get HMG descriptors
        hmgDesc_, hmgInfo_ = Video2DenseHMGVolumes(sampledVid, blockSize, numBlocks, numOr)
        hmgDesc.append(hmgDesc_)
        hmgInfo.append(hmgInfo_)

        # Print statistics
        extractionTimeHMG.append(time.time() - start_time)
        totalDescriptorTime = extractionTimeHMG[-1] + videoReadTime
        print('HMG: frames/block: {} sample rate: {} sec/vid: {:.2f} frame/sec: {:.2f}'.format(
            blockSize[2], frameSampleRate, totalDescriptorTime, vid.shape[2] / totalDescriptorTime))

    return hogDesc, hogInfo, hofDesc, hofInfo, MBHRowDesc, MBHColDesc, mbhInfo, hmgDesc, hmgInfo


### Perform Binning & Normalization

In [8]:
def performBinningAndNormalization(descriptors):
    import numpy as np

    numBins = 64  # Number of bins for histogram binning

    for idx in range(len(descriptors)):
        descriptor = descriptors[idx]

        # Perform binning
        descriptor = np.histogram(descriptor.flatten(), bins=numBins, range=(0, 1))[0]

        # Perform L2 normalization
        descriptor = descriptor / np.linalg.norm(descriptor, ord=2)

        # Update the descriptors
        descriptors[idx] = descriptor

    return descriptors


### Bin & Normalize All Descriptors

In [9]:
def BinAndNormalize(hogDesc, hofDesc, MBHRowDesc, MBHColDesc, hmgDesc):
    hogDescNorm = performBinningAndNormalization(hogDesc)
    hofDescNorm = performBinningAndNormalization(hofDesc)
    mbhxDescNorm = performBinningAndNormalization(MBHRowDesc)
    mbhyDescNorm = performBinningAndNormalization(MBHColDesc)
    hmgDescNorm = performBinningAndNormalization(hmgDesc)

    return hogDescNorm, hofDescNorm, mbhxDescNorm, mbhyDescNorm, hmgDescNorm


### Fisher Vector Encoding

In [10]:
import numpy as np
import cv2
import os
import pickle

def fisherVectorEncoder(vlfeatPath, hogDesc, hogInfo, hofDesc, hofInfo, MBHRowDesc, MBHColDesc, mbhInfo, hmgDesc, hmgInfo):
    # Add VLFeat library to the MATLAB path
    os.environ['PATH'] += os.pathsep + vlfeatPath
    
    # Set the desired size for the descriptors
    desiredSize = hogDesc[0].shape

    # Resize the HOG descriptors to the desired size
    resizedHOGDesc = resizeDescriptors(hogDesc, desiredSize)
    
    # Resize the HOF descriptors to the desired size
    resizedHOFDesc = resizeDescriptors(hofDesc, desiredSize)
    
    # Resize the MBHRow descriptors to the desired size
    resizedMBHRowDesc = resizeDescriptors(MBHRowDesc, desiredSize)
    
    # Resize the MBHCol descriptors to the desired size
    resizedMBHColDesc = resizeDescriptors(MBHColDesc, desiredSize)
    
    # Resize the HMG descriptors to the desired size
    resizedHMGDesc = resizeDescriptors(hmgDesc, desiredSize)

    # Load the resized descriptors from memory
    HOG = np.concatenate(resizedHOGDesc, axis=3).astype(np.float32)
    HOF = np.concatenate(resizedHOFDesc, axis=3).astype(np.float32)
    MBHRow = np.concatenate(resizedMBHRowDesc, axis=3).astype(np.float32)
    MBHCol = np.concatenate(resizedMBHColDesc, axis=3).astype(np.float32)
    HMG = np.concatenate(resizedHMGDesc, axis=3).astype(np.float32)

    # Set the number of clusters
    K = 16

    # Train the GMM for HOG
    HOG_2D = HOG.reshape(-1, HOG.shape[3]).T
    hogMeans, hogCovariances, hogPriors = vl_gmm(HOG_2D, K)

    # Compute the Fisher vector for HOG
    hogFisherVector = vl_fisher(HOG_2D, hogMeans, hogCovariances, hogPriors)

    # Train the GMM for HOF
    HOF_2D = HOF.reshape(-1, HOF.shape[3]).T
    hofMeans, hofCovariances, hofPriors = vl_gmm(HOF_2D, K)

    # Compute the Fisher vector for HOF
    hofFisherVector = vl_fisher(HOF_2D, hofMeans, hofCovariances, hofPriors)

    # Train the GMM for MBHRow
    MBHRow_2D = MBHRow.reshape(-1, MBHRow.shape[3]).T
    mbhRowMeans, mbhRowCovariances, mbhRowPriors = vl_gmm(MBHRow_2D, K)

    # Compute the Fisher vector for MBHRow
    mbhRowFisherVector = vl_fisher(MBHRow_2D, mbhRowMeans, mbhRowCovariances, mbhRowPriors)
    
    # Train the GMM for MBHCol
    MBHCol_2D = MBHCol.reshape(-1, MBHCol.shape[3]).T
    mbhColMeans, mbhColCovariances, mbhColPriors = vl_gmm(MBHCol_2D, K)

    # Compute the Fisher vector for MBHCol
    mbhColFisherVector = vl_fisher(MBHCol_2D, mbhColMeans, mbhColCovariances, mbhColPriors)
    
    # Train the GMM for HMG
    HMG_2D = HMG.reshape(-1, HMG.shape[3]).T
    hmgMeans, hmgCovariances, hmgPriors = vl_gmm(HMG_2D, K)

    # Compute the Fisher vector for HMG
    hmgFisherVector = vl_fisher(HMG_2D, hmgMeans, hmgCovariances, hmgPriors)

    # Save the GMM parameters to disk
    with open('gmm_hog.pkl', 'wb') as f:
        pickle.dump((hogMeans, hogCovariances, hogPriors), f)
    with open('gmm_hof.pkl', 'wb') as f:
        pickle.dump((hofMeans, hofCovariances, hofPriors), f)
    with open('gmm_mbh.pkl', 'wb') as f:
        pickle.dump((mbhRowMeans, mbhRowCovariances, mbhRowPriors, mbhColMeans, mbhColCovariances, mbhColPriors), f)
    with open('gmm_hmg.pkl', 'wb') as f:
        pickle.dump((hmgMeans, hmgCovariances, hmgPriors), f)

    # Save the encoded features to a file
    np.save('hog_fisher_vector.npy', hogFisherVector)
    np.save('hof_fisher_vector.npy', hofFisherVector)
    np.save('mbh_row_fisher_vector.npy', mbhRowFisherVector)
    np.save('mbh_col_fisher_vector.npy', mbhColFisherVector)
    np.save('hmg_fisher_vector.npy', hmgFisherVector)

    # Print the dimensions of each Fisher vector
    print('HOG Fisher vector dimensions:', hogFisherVector.shape)
    print('HOF Fisher vector dimensions:', hofFisherVector.shape)
    print('MBH Row Fisher vector dimensions:', mbhRowFisherVector.shape)
    print('MBH Col Fisher vector dimensions:', mbhColFisherVector.shape)
    print('HMG Fisher vector dimensions:', hmgFisherVector.shape)

    print('Fisher vector encoding done!')

def resizeDescriptors(desc, desiredSize):
    resizedDesc = []
    for d in desc:
        resizedDesc.append(cv2.resize(d, desiredSize))
    return resizedDesc

def vl_gmm(X, K):
    from sklearn.mixture import GaussianMixture
    gmm = GaussianMixture(n_components=K, covariance_type='diag')
    gmm.fit(X)
    means = gmm.means_.T
    covariances = gmm.covariances_.T
    priors = gmm.weights_
    return means, covariances, priors

def vl_fisher(X, means, covariances, priors):
    from fisher_vectors import FisherVectors
    fv = FisherVectors(means, covariances, priors)
    fisher_vector = fv.transform(X)
    return fisher_vector


### Perform Label Encoding

In [11]:
def performLabelEncoding(subjectNames):
    # Unique subject names
    unique_names = list(set(subjectNames))

    # Create a mapping between subject names and integer labels
    label_map = {name: label for label, name in enumerate(unique_names, start=1)}

    # Perform label encoding
    encoded_labels = [label_map[name] for name in subjectNames]

    print('\nEncoded Labels:')
    print(encoded_labels)

    return encoded_labels


### Concatenation

In [12]:
def concatenateFisherVectors(hogFisherVector, hofFisherVector, mbhRowFisherVector, mbhColFisherVector, hmgFisherVector):
    # Concatenate the Fisher vectors
    concatenated_vectors = np.concatenate((hogFisherVector, hofFisherVector, mbhRowFisherVector, mbhColFisherVector, hmgFisherVector), axis=1)

    # Save the concatenated vectors
    np.save('concatenated_vectors.npy', concatenated_vectors)

    # Display the concatenated vectors
    print('\nConcatenated Fisher Vectors:')
    print(concatenated_vectors)

    return concatenated_vectors


### Save Concatenated Vectors With Encoded Labels

In [13]:
def saveConcatWithLabels(matrix, value, seq):
    filename = 'concat_with_labels.npy'

    concatenated_values = np.concatenate((matrix, np.full((matrix.shape[0], 1), value), np.full((matrix.shape[0], 1), seq)), axis=1)

    try:
        existing_matrix = np.load(filename)
        result = np.concatenate((existing_matrix, concatenated_values), axis=0)
    except FileNotFoundError:
        result = concatenated_values

    np.save(filename, result)

    print(result)

### Principal Component Analysis

In [14]:
def allVidsPCA():

    # Load the data from the numpy file
    data = np.load('concat_with_labels.npy')

    # Extract the necessary columns for PCA
    pcaData = data[:, :-2]

    # Perform PCA
    desiredVariance = 0.96
    pca = PCA(n_components=desiredVariance)
    pcaScores = pca.fit_transform(pcaData)

    # Append the last two columns to the PCA scores
    pcaScores = np.concatenate((pcaScores, data[:, -2:]), axis=1)

    # Save the PCA scores to a file
    np.save('allvids_pca_scores.npy', pcaScores)

    print(pcaScores)

### Support Vector Machine

In [15]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np

def run_disp_SVM():
    # Load the PCA scores and labels from the numpy file
    data = np.load('allvids_pca_scores.npy')
    pcaScores = data

    # Separate the last column (testing/training indicator) and second last column (labels)
    testingIndicator = pcaScores[:, -1]
    labels = pcaScores[:, -2]

    # Find the indices for testing and training data
    testingIndices = (testingIndicator == 4)

    # Separate training and testing data
    trainingData = pcaScores[~testingIndices, :-2]
    trainingLabels = labels[~testingIndices]
    testingData = pcaScores[testingIndices, :-2]
    testingLabels = labels[testingIndices]

    # Train the SVM classifier
    model = SVC()
    model.fit(trainingData, trainingLabels)

    # Predict labels for testing data
    predictedLabels = model.predict(testingData)

    # Calculate the accuracy of the SVM classifier
    accuracy = accuracy_score(testingLabels, predictedLabels) * 100

    # Display the accuracy of the SVM classifier
    print('SVM Accuracy:')
    print(accuracy)

    # Print the results for all subjects
    print('Subject Results:')
    subjects = np.unique(testingLabels)
    for subject in subjects:
        subjectIndices = (testingLabels == subject)
        subjectPredictions = predictedLabels[subjectIndices]
        subjectAccuracy = accuracy_score(np.full_like(subjectPredictions, subject), subjectPredictions) * 100
        print('Subject', subject, ':')
        print('Accuracy:', subjectAccuracy)
        print('Predictions:', subjectPredictions)


### Facade

In [16]:
def runFisherConcatPCA(videoPath, subjectName, sequenceName):
    # Load the video and get the read time
    vid, videoReadTime = loadVideo(videoPath)

    # Extract features
    hogDesc, hogInfo, hofDesc, hofInfo, MBHRowDesc, MBHColDesc, mbhInfo, hmgDesc, hmgInfo = extractFeatures(vid, videoReadTime)

    # Performing binning & normalization
    hogDescNorm, hofDescNorm, mbhxDescNorm, mbhyDescNorm, hmgDescNorm = BinAndNormalize(hogDesc, hofDesc, MBHRowDesc, MBHColDesc, hmgDesc)
    
    # Set vlfeat path
    vlfeatPath = 'E:\bsef19m501\vlfeat-0.9.21-bin'

    subjectNames = ['fyc', 'hy', 'ljg', 'lqf', 'lsl', 'ml', 'nhz', 'rj', 'syj', 'wl', 'wq', 'wyc', 'xch', 'xxj', 'yjf', 'zc', 'zdx', 'zjg', 'zl', 'zyf']

    # Perform label encoding
    encoded_labels = performLabelEncoding(subjectNames)

    # Find the encoded label for the given subject name
    encodedLabel = encoded_labels[subjectNames.index(subjectName)]

    print('\nEncoded Label for Current Subject:')
    print(encodedLabel)

    # Perform Fisher Vector Encoding
    hogFisherVector, hofFisherVector, mbhRowFisherVector, mbhColFisherVector, hmgFisherVector = fisherVectorEncoder(vlfeatPath, hogDescNorm, hogInfo, hofDescNorm, hofInfo, mbhxDescNorm, mbhyDescNorm, mbhInfo, hmgDescNorm, hmgInfo)

    # Concatenate after Fisher Vector Encoding
    concatenated_vectors = concatenateFisherVectors(hogFisherVector, hofFisherVector, mbhRowFisherVector, mbhColFisherVector, hmgFisherVector)

    saveConcatWithLabels(concatenated_vectors, encodedLabel, sequenceName)


### Caller Function

In [17]:
def runCode():
    # Iterate over the subject folders
    subjectNames = ['fyc', 'hy', 'ljg', 'lqf', 'lsl', 'ml', 'nhz', 'rj', 'syj', 'wl', 'wq', 'wyc', 'xch', 'xxj', 'yjf', 'zc', 'zdx', 'zjg', 'zl', 'zyf']
    
    for subjectName in subjectNames:
        subjectFolder = subjectName
        print(f'\nSubject Folder: {subjectFolder}\n')
        
        # Iterate over the sequence folders for each subject
        sequenceNames = ['00_1', '00_2', '00_3', '00_4']
        
        for sequenceName in sequenceNames:
            sequenceFolder = f'{subjectFolder}/{sequenceName}'
            print(f'Sequence Folder: {sequenceFolder}\n')
            
            # Construct the video filename
            videoFilename = subjectName + sequenceName
            videoPath = f'{videoFilename}.avi'
            print(f'Video Path: {videoPath}\n')
            completePath = 'Videos\\'+ subjectName + '\\' + sequenceName + '\\' + videoPath
            print(completePath)
            
            # Process the video file as needed
            # print(videoPath)

            # Run complete code
            videoPath = os.path.join(os.getcwd(), completePath )
            print(f'Video Path: {videoPath}\n')
            
            # runCompleteCode(videoPath)

            runFisherConcatPCA(videoPath, subjectName, sequenceName)

            # runNewCode(videoPath, subjectName, sequenceName)


# videoPath = os.path.join(os.getcwd(), 'v_HulaHoop_g11_c04.avi')
# runFisherConcatPCA(videoPath, subjectName, sequenceName)

runCode()



Subject Folder: fyc

Sequence Folder: fyc/00_1

Video Path: fyc00_1.avi

Videos\fyc\00_1\fyc00_1.avi
Video Path: C:\Users\LENOVO\Desktop\AIM\Videos\fyc\00_1\fyc00_1.avi

Using VideoReader from Matlab to load video.


NameError: name 'VideoReadNative' is not defined